## Подсистемы для поиска


## Полнотекстовый поиск и Автодополнение (Typeahead)

### Проблема: `LIKE 'prefix%'`
В SQL запросы с `LIKE` работают медленно на больших объемах ($O(N)$ или $O(\log N)$). Для автодополнения (T9) нужно быстрее.

### Решение: Префиксное дерево (Trie)
Структура данных, где узлы — это буквы. Путь от корня — слово.
*   **Поиск**: $O(L)$, где $L$ — длина слова (очень быстро).
*   **Оптимизация**: Хранить топ-5 популярных запросов прямо в узле дерева.

### Алгоритмы поиска подстрок
1.  **Кнута-Морриса-Пратта (KMP)**: Использует префикс-функцию, чтобы не откатываться назад при несовпадении.
2.  **Ахо-Корасик**: Поиск множества слов одновременно. Строит автомат (Trie + fail links).
3.  **Elasticsearch / Lucene**: Инвертированный индекс (Inverted Index). Слово -> Список документов.

---

## Гео-поиск (Geo-spatial Search)
Задача: Найти ближайшие точки (Uber, Tinder, Maps).
Запрос `WHERE lat BETWEEN ... AND long BETWEEN ...` медленный.

### 1. GeoHash
Разбиваем карту на сетку. Каждая ячейка имеет строковый ID.
*   Чем длиннее строка, тем точнее квадрат.
*   Соседние квадраты часто имеют общий префикс.
*   Поиск: `LIKE 'w21z%'` (быстро по индексу).

### 2. QuadTree (Дерево квадрантов)
Рекурсивно делим квадрат на 4 части, пока в квадрате не станет мало точек (например, < 100).
*   В памяти (RAM) строим дерево.
*   Быстрый поиск k-ближайших соседей (k-NN).
*   Нужно перестраивать при перемещении точек.


## Практика: Префиксное дерево (Trie)


### Для чего?
Для автодополнения (Typeahead). Позволяет мгновенно находить слова, начинающиеся на `pre...`.
Каждый узел - буква. Спускаясь по веткам, мы собираем слово.


In [ ]:

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search_prefix(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return [] # Префикс не найден
            node = node.children[char]
        
        # Собираем все слова, которые продолжаются из этого узла
        results = []
        self._dfs(node, prefix, results)
        return results

    def _dfs(self, node, current_prefix, results):
        if node.is_end_of_word:
            results.append(current_prefix)
        
        for char, child_node in node.children.items():
            self._dfs(child_node, current_prefix + char, results)

# --- Тест ---
trie = Trie()
words = ["apple", "app", "application", "apricot", "banana", "batman"]
for w in words:
    trie.insert(w)

query = "app"
print(f"Запрос: '{query}'")
print(f"Автодополнение: {trie.search_prefix(query)}")

query2 = "ba"
print(f"Запрос: '{query2}'")
print(f"Автодополнение: {trie.search_prefix(query2)}")


## Практика: GeoHash (Упрощенный)


### Идея
Превратить 2D координаты (широту и долготу) в 1D строку (хеш).
Мы делим карту пополам: лево/право (долгота), верх/низ (широта).
0 - меньше среднего, 1 - больше среднего.
Чередуя биты долготы и широты, получаем уникальный бинарный код зоны.


In [ ]:

def encode_geohash(lat, lon, precision=5):
    # Диапазоны координат (Земля)
    lat_range = (-90.0, 90.0)
    lon_range = (-180.0, 180.0)
    
    # Символы Base32 для кодирования
    base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    
    bits = []
    # 5 бит на 1 символ, precision символов
    total_bits = precision * 5
    
    for i in range(total_bits):
        # Чередуем: четные биты - долгота (lon), нечетные - широта (lat)
        if i % 2 == 0:
            mid = (lon_range[0] + lon_range[1]) / 2
            if lon > mid:
                bits.append('1')
                lon_range = (mid, lon_range[1])
            else:
                bits.append('0')
                lon_range = (lon_range[0], mid)
        else:
            mid = (lat_range[0] + lat_range[1]) / 2
            if lat > mid:
                bits.append('1')
                lat_range = (mid, lat_range[1])
            else:
                bits.append('0')
                lat_range = (lat_range[0], mid)
                
    # Склеиваем биты и переводим в Base32
    binary_string = "".join(bits)
    hash_string = ""
    for i in range(0, len(binary_string), 5):
        chunk = binary_string[i:i+5]
        index = int(chunk, 2)
        hash_string += base32[index]
        
    return hash_string

# --- Тест ---
lat, lon = 55.7558, 37.6173 # Москва
gh = encode_geohash(lat, lon, precision=6)
print(f"Москва ({lat}, {lon}) -> GeoHash: {gh}")

lat2, lon2 = 55.7522, 37.6156 # Кремль (совсем рядом)
gh2 = encode_geohash(lat2, lon2, precision=6)
print(f"Кремль ({lat2}, {lon2}) -> GeoHash: {gh2}")
print(f"Совпадение префикса: {gh[:4] == gh2[:4]} (Значит они рядом!)")
